In [5]:
import sys

In [6]:
sys.path

['/Users/thomassiegert/data/SPI/GRBs/120711A',
 '/Users/thomassiegert/MEGAlib/MEGAlib/external/root_v6.22.02/lib',
 '/usr/local/opt/python@3.7/Frameworks/Python.framework/Versions/3.7/lib/python37.zip',
 '/usr/local/opt/python@3.7/Frameworks/Python.framework/Versions/3.7/lib/python3.7',
 '/usr/local/opt/python@3.7/Frameworks/Python.framework/Versions/3.7/lib/python3.7/lib-dynload',
 '',
 '/Users/thomassiegert/.virtualenvs/3ML/lib/python3.7/site-packages',
 '/Users/thomassiegert/.virtualenvs/3ML/lib/python3.7/site-packages/IPython/extensions',
 '/Users/thomassiegert/.ipython']

In [9]:
from pathlib import Path

import numba as nb
import numpy as np
import scipy.stats as stats

from natsort import natsorted

import matplotlib.pyplot as plt


from threeML import *

#plt.style.use("thomas")
import warnings
warnings.simplefilter("ignore")


%matplotlib widget



In [12]:
spi = OGIPLike("spi",
               observation="fit_GRB120711A_BB_SwiftPosition_spec/spectra_GRB_OGIP.fits",
               response="fit_GRB120711A_BB_SwiftPosition_spec/spectral_response.rmf_OGIP.fits")



[WARNING ] Found TSTOP and TELAPSE. This file is invalid. Using TSTOP.
[WARNING ] FILTER is not set. This is not a compliant OGIP file. Assuming no FILTER.
[WARNING ] The response is in an extension called MATRIX, which usually means you also need an ancillary file (ARF) which you didn't provide. You should refer to the documentation  of the instrument and make sure you don't need an ARF.
[INFO    ] Auto-probed noise models:
[INFO    ] - observation: gaussian
[INFO    ] - background: None


In [13]:
spi.view_count_spectrum();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
band = Band()
band.alpha.prior = Truncated_gaussian(lower_bound=-1.5, upper_bound=1, mu=-1, sigma=0.5)
band.beta.prior = Truncated_gaussian(lower_bound=-5, upper_bound=-1.6, mu=-2, sigma=0.5)
band.xp.prior = Log_normal(mu=2, sigma=1)
band.xp.bounds = (0, None)
band.K.prior = Log_uniform_prior(lower_bound=1e-10, upper_bound=1e3)
ps = PointSource("grb", 0, 0, spectral_shape=band)
band_model = Model(ps)

In [15]:
bayes = BayesianAnalysis(band_model, DataList(spi))

In [19]:
bayes.set_sampler()
bayes.sampler.setup(n_live_points=400,n_iterations=2000)
_ = bayes.sample()

[INFO    ] using default sampler emcee
[INFO    ] sampler is setup with default parameters


  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

[INFO    ] Mean acceptance fraction: 0.57735
Maximum a posteriori probability (MAP) point:



,result,unit
parameter,,
grb.spectrum.main.Band.K,(3.03 +/- 0.12) x 10^-2,1 / (cm2 keV s)
grb.spectrum.main.Band.alpha,(-9.1 +/- 0.5) x 10^-1,
grb.spectrum.main.Band.xp,(1.30 +/- 0.25) x 10^3,keV
grb.spectrum.main.Band.beta,-2.5 +/- 0.5,



Values of -log(posterior) at the minimum:



,-log(posterior)
spi,-20.664508
total,-20.664508



Values of statistical measures:



,statistical measures
AIC,50.810497
BIC,55.191960
DIC,49.129311
PDIC,3.794007


In [20]:
bayes.restore_median_fit()

display_spectrum_model_counts(bayes);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
plot_spectra(bayes.results, flux_unit="erg2/(cm2 s keV)");

processing MLE analyses: 0it [00:00, ?it/s]

processing Bayesian analyses:   0%|          | 0/1 [00:00<?, ?it/s]

Propagating errors:   0%|          | 0/100 [00:00<?, ?it/s]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
bayes.results.corner_plot();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …